In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from copy import copy
idx = pd.IndexSlice

In [3]:
with open('../data/all_declarations.json', 'r') as f:
    data = json.load(f)

In [4]:
data['data'][0]

{'date': '31/_12/2016г.',
 'deputy': {'income': ['997 146 с. -по месту основной работы',
   '714 546 с.-пенсия'],
  'other_estate': [],
  'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 172}],
  'spendeturs': []},
 'relatives': {'income': [],
  'other_estate': [{'наименование': 'лег/автомобиль ',
    'сведения о транспортных средствах': 'Тойота Камри, 2015г. V=2494Б'}],
  'real_estate': [{'наименование': 'квартира', 'площадь (кв.м)': 187}],
  'spendeturs': []},
 'Должность декларанта': 'Депутат',
 'Наименование государственного/ муниципального органа': 'Жогорку Кенеш Кыргызской Республики',
 'ФИО декларанта': 'Абдылдаев Мыктыбек Юсупович'}

In [5]:
json_normalize(data['data']).head(5)

,date,deputy.income,deputy.other_estate,deputy.real_estate,deputy.spendeturs,relatives.income,relatives.other_estate,relatives.real_estate,relatives.spendeturs,Должность декларанта,Наименование государственного/ муниципального органа,ФИО декларанта
0,31/_12/2016г.,"[997 146 с. -по месту основной работы, 714 546...",[],"[{'наименование': 'дом', 'площадь (кв.м)': 172}]",[],[],"[{'наименование': 'лег/автомобиль ', 'сведения...","[{'наименование': 'квартира', 'площадь (кв.м)'...",[],Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович
1,31/_12/2016г.,[787 075 с. - по месту основной работы ],"[{'наименование': 'автотранспортное средство',...",[],[],[],[],[],[],Депутат,Жогорку Кенеш Кыргызской Республики,Абжалиев Алиярбек Токобекович
2,31/_12/2016г.,[597 633 с. - по месту основной работы],[],[],[],[],"[{'наименование': 'лег/автомобиль', 'сведения ...","[{'наименование': 'квартира ', 'площадь (кв.м)...",[],Депутат,Жогорку Кенеш Кыргызской Республики,Айнакулов Мухтарбек Айнакулович
3,31/_12/2016г.,"[786 243 с. - по месту основной работы, 198 33...",[],"[{'наименование': 'нежилое помещение', 'площад...",[],[],"[{'наименование': 'автомобиль ', 'сведения о т...","[{'наименование': 'дом ', 'площадь (кв.м)': 32...",[],Депутат,Жогорку Кенеш Кыргызской Республики,Айдаров Салайдин Абдираевич
4,31/_12/2016г.,[852 417 с. - по месту основной работы ],[],"[{'наименование': 'земельный участок', 'площад...",[],[],"[{'наименование': 'автомобиль', 'сведения о тр...",[],[],Депутат,Жогорку Кенеш Кыргызской Республики,Акматов Алмасбек Жумабекович


## custom unroll

In [6]:
x = data['data'][0]
x

{'date': '31/_12/2016г.',
 'deputy': {'income': ['997 146 с. -по месту основной работы',
   '714 546 с.-пенсия'],
  'other_estate': [],
  'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 172}],
  'spendeturs': []},
 'relatives': {'income': [],
  'other_estate': [{'наименование': 'лег/автомобиль ',
    'сведения о транспортных средствах': 'Тойота Камри, 2015г. V=2494Б'}],
  'real_estate': [{'наименование': 'квартира', 'площадь (кв.м)': 187}],
  'spendeturs': []},
 'Должность декларанта': 'Депутат',
 'Наименование государственного/ муниципального органа': 'Жогорку Кенеш Кыргызской Республики',
 'ФИО декларанта': 'Абдылдаев Мыктыбек Юсупович'}

In [7]:
x['deputy']

{'income': ['997 146 с. -по месту основной работы', '714 546 с.-пенсия'],
 'other_estate': [],
 'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 172}],
 'spendeturs': []}

In [8]:
def unroll_declaration(d):
    results = []
    l1, l2 = ['deputy', 'relatives'], ['income', 'spendeturs', 'real_estate', 'other_estate']
    
    meta = {k:v for k,v in d.items() if k not in l1}
    
    for person in l1:
        data = d[person]
        
        for tp in l2:
            for el in data[tp]:
                try:
                    record = copy(el)
                    if tp in ('income', 'spendeturs'):
                        record = {'income': record}

                    record['type'] = tp
                    record['person'] = person
                    record.update(meta)
                    results.append(record)
                except Exception as inst:
                    raise Exception(tp, el, inst)

    return results    

In [9]:
declaration_records = []

for i, el in enumerate(data['data']):
    print(i, el['ФИО декларанта'])
    declaration_records.extend(unroll_declaration(el))

0 Абдылдаев Мыктыбек Юсупович
1 Абжалиев Алиярбек Токобекович
2 Айнакулов Мухтарбек Айнакулович
3 Айдаров Салайдин Абдираевич
4 Акматов Алмасбек Жумабекович
5 Акаев Жанарбек Кубанычович
6 Алимбеков Нурбек Каарыевич
7 Алтыбаева Айнуру Тойчиевна
8 Алтынбеков Айбек Алтынбекович
9 Аманкулов Марат Аскерович
10 Арапбаев Азамат Абдуллажанович
11 Артыкбаев Осмонбек Мамбетжанович
12 Асылбек уулу Дамирбек
13 Асылбаева Гюльшат Кадыровна
14 Артыков Анвар Артыкович
15 Аттокуров Данияр Бексултанович
16 Баатырбеков Алмазбек Баатырбекович
17 Бакиров Мирлан Исакбекович
18 Байбакпаев Экмат Джурукпаевич
19 Баймуратов Калдарбек Шакирович
20 Бабанов Омурбек Токтогулович
21 Бакчиев Джаныбек Абдукапарович
22 Баяхунов Нурмухаммат  Баяхунович
23 Бекешев Дастан Далабайевич
24 Бокоев Кенжебек Сатымкулович
25 Джумабеков Дастанбек Артисбекович
26 Джакупова Чолпон Идиновна
27 Жамалдинов Зиядин Исламович
28 Джураев Сайдолимжон Маруфович
29 Жамангулов Акылбек Жекшенович
30 Жапаров Акылбек Усенбекович
31 Жеенчороев Ми

In [10]:
df = pd.DataFrame(declaration_records)

In [11]:
df.head(10)

,date,income,person,type,Должность декларанта,Наименование государственного/ муниципального органа,ФИО декларанта,доля (%),наименование,площадь (кв.м),сведения о транспортных средствах,хозяйствующий субъект
0,31/_12/2016г.,997 146 с. -по месту основной работы,deputy,income,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович,NaN,NaN,NaN,NaN,NaN
1,31/_12/2016г.,714 546 с.-пенсия,deputy,income,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович,NaN,NaN,NaN,NaN,NaN
2,31/_12/2016г.,NaN,deputy,real_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович,NaN,дом,172,NaN,NaN
3,31/_12/2016г.,NaN,relatives,real_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович,NaN,квартира,187,NaN,NaN
4,31/_12/2016г.,NaN,relatives,other_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Абдылдаев Мыктыбек Юсупович,NaN,лег/автомобиль,NaN,"Тойота Камри, 2015г. V=2494Б",NaN
5,31/_12/2016г.,787 075 с. - по месту основной работы,deputy,income,Депутат,Жогорку Кенеш Кыргызской Республики,Абжалиев Алиярбек Токобекович,NaN,NaN,NaN,NaN,NaN
6,31/_12/2016г.,NaN,deputy,other_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Абжалиев Алиярбек Токобекович,NaN,автотранспортное средство,NaN,"Тойота Лендкрузер, 2012г., V=4",NaN
7,31/_12/2016г.,597 633 с. - по месту основной работы,deputy,income,Депутат,Жогорку Кенеш Кыргызской Республики,Айнакулов Мухтарбек Айнакулович,NaN,NaN,NaN,NaN,NaN
8,31/_12/2016г.,NaN,relatives,real_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Айнакулов Мухтарбек Айнакулович,NaN,квартира,115,NaN,NaN
9,31/_12/2016г.,NaN,relatives,other_estate,Депутат,Жогорку Кенеш Кыргызской Республики,Айнакулов Мухтарбек Айнакулович,NaN,лег/автомобиль,NaN,"Тойота РАФ 4, 2004г., V=1998",NaN


In [12]:
df.columns

Index(['date', 'income', 'person', 'type', 'Должность декларанта',
       'Наименование государственного/ муниципального органа',
       'ФИО декларанта', 'доля (%)', 'наименование', 'площадь (кв.м)',
       'сведения о транспортных средствах', 'хозяйствующий субъект'],
      dtype='object')

In [13]:
new_col_names = {'Наименование государственного/ муниципального органа': 'department',
                 'ФИО декларанта': 'name',
                 'доля (%)': 'share',
                 'наименование': 'details',
                 'площадь (кв.м)':'area_sqm',
                 'сведения о транспортных средствах':"transport-details",
                 "хозяйствующий субъект": "manager"}

df = df.rename(columns=new_col_names)

In [14]:
new_df = df.set_index(['name', 'person','type', df.index])

In [15]:
new_df.head(20)

date  \
name                            person    type                             
Абдылдаев Мыктыбек Юсупович     deputy    income       0   31/_12/2016г.   
                                                       1   31/_12/2016г.   
                                          real_estate  2   31/_12/2016г.   
                                relatives real_estate  3   31/_12/2016г.   
                                          other_estate 4   31/_12/2016г.   
Абжалиев Алиярбек Токобекович   deputy    income       5   31/_12/2016г.   
                                          other_estate 6   31/_12/2016г.   
Айнакулов Мухтарбек Айнакулович deputy    income       7   31/_12/2016г.   
                                relatives real_estate  8   31/_12/2016г.   
                                          other_estate 9   31/_12/2016г.   
Айдаров Салайдин Абдираевич     deputy    income       10  31/_12/2016г.   
                                                       11  31/_12/2016г.   
                                                       12  31/_12/2016г.   
                                          real_estate  13  31/_12/2016г.   
                                                       14  31/_12/2016г.   
                                                       15  31/_12/2016г.   
                                                       16  31/_12/2016г.   
                                relatives real_estate  17  31/_12/2016г.   
                                                       18  31/_12/2016г.   
                                                       19  31/_12/2016г.   

                                                                                                  income  \
name                            person    type                                                             
Абдылдаев Мыктыбек Юсупович     deputy    income       0            997 146 с. -по месту основной работы   
                                                       1                               714 546 с.-пенсия   
                                          real_estate  2                                             NaN   
                                relatives real_estate  3                                             NaN   
                                          other_estate 4                                             NaN   
Абжалиев Алиярбек Токобекович   deputy    income       5          787 075 с. - по месту основной работы    
                                          other_estate 6                                             NaN   
Айнакулов Мухтарбек Айнакулович deputy    income       7           597 633 с. - по месту основной работы   
                                relatives real_estate  8                                             NaN   
                                          other_estate 9                                             NaN   
Айдаров Салайдин Абдираевич     deputy    income       10          786 243 с. - по месту основной работы   
                                                       11                            198 336 с. - пенсия   
                                                       12  1 000 000 с.-от гражданско-правовых договоров   
                                          real_estate  13                                            NaN   
                                                       14                                            NaN   
                                                       15                                            NaN   
                                                       16                                            NaN   
                                relatives real_estate  17                                            NaN   
                                                       18                                            NaN   
                                                       19                                            NaN   

                  

## Parse everything

#### 1. Parse dates

In [16]:
new_df['date'] = pd.to_datetime(new_df['date'], format='%d/_%m/%Yг.')

## Income and spenditures

In [17]:
## simple but might be questionable 

new_df['income_details'] = new_df['income'].str.extract(r'^(?:[\d| |,]+)(.*)')
new_df['income_amount'] = new_df['income'].str.extract(r'^([\d| |,]+).*')

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()
/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [18]:
new_df.loc[idx[:, :, 'spendeturs'], :].head(10)

date  \
name                             person    type                        
Арапбаев Азамат Абдуллажанович   relatives spendeturs 91  2016-12-31   
                                                      92  2016-12-31   
                                                      93  2016-12-31   
                                                      94  2016-12-31   
Бабанов Омурбек Токтогулович     relatives spendeturs 177 2016-12-31   
Жеенчороев Мирлан Канатбекович   relatives spendeturs 266 2016-12-31   
Жээнбеков Асылбек Шарипович      relatives spendeturs 342 2016-12-31   
Зулпукаров Торобай Закирович     relatives spendeturs 351 2016-12-31   
Ибраимжанов Болотбек Сапарбаевич relatives spendeturs 364 2016-12-31   
Мадеминов Мурадыл Ганыевич       relatives spendeturs 521 2016-12-31   

                                                                   income  \
name                             person    type                             
Арапбаев Азамат Абдуллажанович   relatives spendeturs 91      330 800 с.    
                                                      92    59 440 953 с.   
                                                      93     4 992 791 с.   
                                                      94        62 000 с.   
Бабанов Омурбек Токтогулович     relatives spendeturs 177  272 756 000 с.   
Жеенчороев Мирлан Канатбекович   relatives spendeturs 266      800 000 с.   
Жээнбеков Асылбек Шарипович      relatives spendeturs 342    1 768 000 с.   
Зулпукаров Торобай Закирович     relatives spendeturs 351      897 000 с.   
Ибраимжанов Болотбек Сапарбаевич relatives spendeturs 364   3 652 436 с.    
Мадеминов Мурадыл Ганыевич       relatives spendeturs 521    3 000 000 с.   

                                                          Должность декларанта  \
name                             person    type                                  
Арапбаев Азамат Абдуллажанович   relatives spendeturs 91               Депутат   
                                                      92               Депутат   
                                                      93               Депутат   
                                                      94               Депутат   
Бабанов Омурбек Токтогулович     relatives spendeturs 177              Депутат   
Жеенчороев Мирлан Канатбекович   relatives spendeturs 266              депутат   
Жээнбеков Асылбек Шарипович      relatives spendeturs 342              Депутат   
Зулпукаров Торобай Закирович     relatives spendeturs 351              Депутат   
Ибраимжанов Болотбек Сапарбаевич relatives spendeturs 364              Депутат   
Мадеминов Мурадыл Ганыевич       relatives spendeturs 521              Депутат   

                                                                                    department  \
name                             person    type                                                  
Арапбаев Азамат Абдуллажанович   relatives spendeturs 91   Жогорку Кенеш Кыргызской Республики   
                                                      92   Жогорку Кенеш Кыргызской Республики   
                                                      93   Жогорку Кенеш Кыргызской Республики   
                                                      94   Жогорку Кенеш Кыргызской Республики   
Бабанов Омурбек Токтогулович     relatives spendeturs 177  Жогорку Кенеш Кыргызской Республики   
Жеенчороев Мирлан Канатбекович   relatives spendeturs 266  Жогорку Кенеш Кыргызской Республики   
Жээнбеков Асылбек Шарипович      relatives spendeturs 342  Жогорку Кенеш Кыргызской Республики   
Зулпукаров Торобай Закирович     relatives spendeturs 351  Жогорку Кенеш Кыргызской Республики   
Ибраимжанов Болотбек Сапарбаевич relatives spendeturs 364  Жогорку Кенеш Кыргызской Республики   
Мадеминов Мурадыл Ганыевич       relatives spendeturs 521  Жогорку Кенеш Кыргызской Республики   

                                                          share details  \
name                             pers

In [21]:
new_df.drop('income', axis=1, inplace=True)

In [23]:
new_df.to_csv('../data/clean_declarations_table.csv')